<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 组合链

In [ ]:
from videos.walkthroughs import walkthrough_23 as walkthrough

In [ ]:
walkthrough()

在这个 notebook 中，您将学习如何组合多个 LLM 的链。

---

## 目标

完成这个 notebook 后，您将：

- 学会如何将链组合起来
- 运用您的能力来链接有意义的语言任务。

---

## 导入

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel

---

## 创建模型实例

In [ ]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## 组合多个 LLM 链

如果您还记得，运行时可以组合成链，但链本身也是运行时。因此，链可以用于组合更大的链。

很容易想象出需要多次调用 LLM 来获得所需结果的任务。我们将从这样的场景开始探索链的组合，管道将一个链的输出传递给下一个链。

为此，我们将使用以下的 `thesis_statements` 列表。请注意，您在这些论题中看到的任何拼写错误都是有意为之的。

In [ ]:
thesis_statements = [
    "The fundametal concepts quantum physcis are difficult to graps, even for the mostly advanced students.",
    "Einstein's theroy of relativity revolutionised undrstanding of space and time, making it clear that they are interconnected.",
    "The first law of thermodynmics states that energy cannot be created or destoryed, excepting only transformed from one form to another.",
    "Electromagnetism is one of they four funadmental forces of nature, and it describes the interaction between charged particles.",
    "In the study of mechanic, Newton's laws of motion provide a comprehensive framework for understading the movement of objects under various forces."
]

我们的目标是将这些论题扩展为一个写好的段落，论题本身将是第一段。然而，您可能已经注意到，这些论题中包含需要纠正的拼写或语法错误。

因此，首先创建一个链来解决拼写和语法问题，然后将更正后的论题链入第二个负责生成完整段落的 LLM 链。

---

## 练习：创建拼写和语法链

首先，创建一个 `grammar_chain`，它在对输入进行拼写和语法纠正后返回输入。

我们在上面已经定义了一个 LLM 实例（`llm`），但您需要创建一个提示模板和输出解析器以包含到链中。

您可能需要逐步开发您的提示模板。确保链只返回纠正后的文本，而不是模型的任何附加内容。

通过将上面定义的 `thesis_statements` 批量发送给您的链来测试它。

如果遇到困难，请查看下面的参考答案。

### 您的代码

In [ ]:
grammar_chain = 'TODO' # TODO: grammar_chain should return its inputs after performing spelling and grammar on them.

### 参考答案

我们首先为拼写和语法纠正设计一个提示。需要特别注意提示的具体性，确保模型只生成纠正后的文本，不包含额外的评论或前言。

In [ ]:
spelling_and_grammar_template = ChatPromptTemplate.from_template("""Fix any spelling or grammatical issues in the following text. Return \
back the correct text and only the corrected text with no additional comment or preface. Text: {text}""")

接下来，我们创建一个字符串输出解析器的实例。

In [ ]:
parser = StrOutputParser()

剩下的就是组合这个链...

In [ ]:
grammar_chain = spelling_and_grammar_template | llm | parser

...并将论题批量传给它。

In [ ]:
corrected_texts = grammar_chain.batch(thesis_statements)

从纠正后的输出来看，模型表现得非常出色。

In [ ]:
for corrected_text in corrected_texts:
    print(corrected_text)

---

## 练习：创建段落生成器链

创建第二个链 `paragraph_generator_chain`。给定一个句子作为输入，将该句子用作生成段落的第一句。

**注意：**这个链不应该包含任何语法或拼写检查功能，只负责生成段落。

通过将上面定义的 `thesis_statements` 批量发送给您的链来测试它。

如果遇到困难，请随时查看下面的*参考答案*。

### 您的代码

In [ ]:
paragraph_generator_chain = 'TODO'

### 参考答案

从设计提示词开始。

In [ ]:
paragraph_generator_template = ChatPromptTemplate.from_template("""Generate a 4 to 8 sentence paragraph that begins with the following \
thesis statement. Return back the paragraph and only the paragrah with no addional comment or preface. Thesis statement: {thesis}""")

由于我们已经有了模型实例和解析器，接下来只需组合这个链...

In [ ]:
paragraph_generator_chain = paragraph_generator_template | llm | parser

...然后将批量的论题发给它。

In [ ]:
paragraphs = paragraph_generator_chain.batch(thesis_statements)

从生成的段落来看，模型表现得非常不错。

值得注意的是，尽管我们没有提示模型去处理拼写和语法错误，但它还是修正了一些拼写错误。不过，显然我们传入的论题中的大部分语法错误依旧存在。

In [ ]:
for paragraph in paragraphs:
    print(paragraph+'\n')

---

## 练习：创建链的链

复用您已经创建的链，创建一个 `corrected_generator_chain`，该链首先使用 LLM 对 `thesis_statements` 进行拼写和语法纠正，然后基于（纠正后的）论题生成完整的段落。

不需要想得太复杂。只要记住，链是运行时，可以像其它运行时一样用管道组合。

通过将上述定义的 `thesis_statements` 批量发送给您的链来测试它。

如果遇到困难，请随时查看下面的*参考答案*。

### 您的代码

### 参考答案

创建更大链所需做的就是将我们已经创建的两个链组合在一起。

In [ ]:
corrected_generator_chain = grammar_chain | paragraph_generator_chain

为了有趣一点，我们可以看看新链的计算图。

In [ ]:
print(corrected_generator_chain.get_graph().draw_ascii())

我们可以像对待较小的链一样，将论题批量发送到这个更大的链。

In [ ]:
paragraphs = corrected_generator_chain.batch(thesis_statements)

从最终输出，我们可以看到段落生成得很好，而且原始论题中的所有拼写和语法错误也都得到了纠正。

In [ ]:
for paragraph in paragraphs:
    print(paragraph+'\n')

---

## 总结

在这个 notebook 中，您学习了如何将链视为运行时并组合起来，现在能多次利用 LLM 来完成所面临的任务。

下一个 notebook 将继续探讨链的组合，但会将重点放在创建和使用并行链的能力上。